# Projeto 1 - Ciência dos Dados

Nome: Rafael Meyer Kahn<br>
Nome: Hélio Zaia Franciscon

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tweepy
import math
import pandas as pd
import json
from random import shuffle
import emoji_list
import emoji

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\lobda\Desktop\Insper\2º Semestre\Ciência dos Dados\DP\Projeto-1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [16]:
dados = pd.read_excel('Netflix.xlsx')
teste = pd.read_excel('Netflix.xlsx', 'Teste')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto abordado durante este projeto foi Netflix. Por ser uma provedora global de filmes e séries, existem diversos tweets sobre seu conteúdo, portanto foi levado em consideração 4 níveis de classificação: Irrelevante, Elogio, Crítica e Sugestão. Com isso, pode-se classificar os tweets obtidos nessas 4 categorias para melhor organizar o tipo de comentário que estão fazendo sobre o produto na internet, resultando assim numa melhor distribuição dos tweets para cada área da empresa analisar os fatores relevantes que estão sendo tweetados on-line.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [4]:
#Exibindo tweets
dados.head(10)

,Treinamento,Classificacao
0,"achei uma série de investigação policial, agor...",1
1,uma sequência do clássico está programada para...,0
2,#alteredcarbon animação da netflix ganha trail...,0
3,"sofá, breja e netflix \nsó na cama vários hits",1
4,o surto que confundi e corri na netflix achand...,0
5,troco em dobro (2020) | nada mais do que puro ...,0
6,netflix não conte comigo para mais nada eu me ...,0
7,rt @hgirardi98: netflix favor parar de lançar ...,2
8,rt @solteirasince: meu hobby é ficar duas hora...,0
9,nao são nem 9h da manhã e já to querendo ir pr...,0


In [5]:
#Definindo uma função que limpa caractéres indesejados
def clean(tabela, titulo):
    a = tabela[titulo]
    a = a.str.lower()

    itens = ['.', ':', ';', '"', "'", '?', '(', ')', '[',']',',', '\n', '\t','*','|','+']
    i2 = ['?', '!']
    emojis = emoji_list.all_emoji
    #separa emojis de palavras
    for emoji in emojis:
        try:
            a = a.str.replace(emoji," {} ".format(emoji) )
        except:
            pass
    #retira caracteres indesejados
    for e in itens:
        a = a.str.replace(e,'')
    #separa '?' e '!' de palavras
    for e in i2:
        a = a.str.replace(e,' {} '.format(e))

    #Substitui caracteres com acento ou variação por caracteres desejados
    a = a.str.replace('é','e')
    a = a.str.replace('ê','e')
    a = a.str.replace('á','a')
    a = a.str.replace('ã','a')
    a = a.str.replace('ô','o')
    a = a.str.replace('ó','o')
    a = a.str.replace('ú','u')
    a = a.str.replace('ç','c')
    a = a.str.replace('í','i')
    a = a.str.replace('@',' @')
        
    tabela[titulo] = a

    return tabela

In [6]:
#Limpando os dados
filtro = clean(dados, 'Treinamento')
#Testando
filtro.head(10)

,Treinamento,Classificacao
0,achei uma serie de investigacao policial agora...,1
1,uma sequencia do classico esta programada para...,0
2,# alteredcarbon animacao da netflix ganha tra...,0
3,sofa breja e netflix so na cama varios hits,1
4,o surto que confundi e corri na netflix achand...,0
5,troco em dobro 2 0 2 0 nada mais do que ...,0
6,netflix nao conte comigo para mais nada eu me ...,0
7,rt @hgirardi 9 8 netflix favor parar de lan...,2
8,rt @solteirasince meu hobby e ficar duas hora...,0
9,nao sao nem 9 h da manha e ja to querendo ir ...,0


In [7]:
#Separando em tabelas por classificação
irrelevante = filtro.loc[(filtro.Classificacao == 0)]
elogio = filtro.loc[(filtro.Classificacao == 1)]
critica = filtro.loc[(filtro.Classificacao == 2)]
sugestao = filtro.loc[(filtro.Classificacao == 3)]

In [8]:
#Definindo uma função que separa as palavras
def tt_split(tabela, titulo):
    a = tabela[titulo].str.split(' ')

    li = pd.DataFrame()
    li['palavras'] = []

    for lista in a:
        for e in ['#','' , ' ']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            if '@' in palavra and palavra != '@':
                lista.remove(palavra) 
            if palavra[:4] == 'http':
                lista.remove(palavra)
        li = li.append({'palavras': lista}, ignore_index=True)
                
    return li

In [9]:
#Separando os tweets das tabelas limpas
irrelevante_limpa = tt_split(irrelevante, 'Treinamento')
elogio_limpa = tt_split(elogio, 'Treinamento')
critica_limpa = tt_split(critica, 'Treinamento')
sugestao_limpa = tt_split(sugestao, 'Treinamento')
#Testando
elogio_limpa.head(10)

,palavras
0,"[achei, uma, serie, de, investigacao, policial..."
1,"[sofa, breja, e, netflix, so, na, cama, varios..."
2,"[estou, assistindo, uma, serie, mt, boa, sobre..."
3,"[rt, a, historia, do, lil, peep, e, incrivel, ..."
4,"[acabei, de, assistir, a, animacao, de, pokemo..."
5,"[rt, melhor, date, do, mundo, e, em, casa, dei..."
6,"[almocar, a, ver, um, filme, random, da, netfl..."
7,"[meu, deus, agora, que, eu, vi, que, tem, itae..."
8,"[e, a, netflix, que, vai, lancar, uma, serie, ..."
9,"[a, netflix, ate, que, tem, um, acervo, bom, d..."


In [10]:
#Definindo função que cria tabela com as palavras separadas
def palavras(tabela):
    a = tabela['palavras']
    prob = pd.DataFrame()
    prob['palavras'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'palavras': e}, ignore_index=True)
    
    return prob['palavras']

In [11]:
#Separando as palavras
irrelevante_separada = palavras(irrelevante_limpa)
elogio_separada = palavras(elogio_limpa)
critica_separada = palavras(critica_limpa)
sugestao_separada = palavras(sugestao_limpa)
#Testando
elogio_separada.head(10)

0           achei
1             uma
2           serie
3              de
4    investigacao
5        policial
6           agora
7              eu
8           nunca
9            mais
Name: palavras, dtype: object

In [12]:
#Obtendo as frequências absolutas
freq_irrelevante = irrelevante_separada.value_counts()
freq_elogio = elogio_separada.value_counts()
freq_critica = critica_separada.value_counts()
freq_sugestao = sugestao_separada.value_counts()
#Testando
freq_elogio.head(10)

netflix    111
e           78
a           58
de          52
que         50
na          46
eu          36
um          28
o           28
da          25
Name: palavras, dtype: int64

In [13]:
#Obtendo as frequências relativas
freq_rel_irrelevante = irrelevante_separada.value_counts(True)
freq_rel_elogio = elogio_separada.value_counts(True)
freq_rel_critica = critica_separada.value_counts(True)
freq_rel_sugestao = sugestao_separada.value_counts(True)
#Testando
freq_rel_elogio.head(10)

netflix    0.055005
e          0.038652
a          0.028741
de         0.025768
que        0.024777
na         0.022795
eu         0.017839
um         0.013875
o          0.013875
da         0.012389
Name: palavras, dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [17]:
teste['Bot'] = 0
teste['Resultado'] = pd.Series()
teste_clean = clean(teste,'Teste')
teste_f = tt_split(teste_clean,'Teste')

#função para multiplicar itens dentro de uma lista
def multiplica(Lista) :     
    u = 1
    for x in Lista: 
         u = u * x  
    return u  

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**